In [1]:
!pip3 install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 15.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488513 sha256=8af26040a1dcff53e73b6a20e2c005efb5a2a0d638585cb670010aaf1b8d1fd8
  Stored in directory: /home/ubuntu/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
!pip3 install findspark

Defaulting to user installation because normal site-packages is not writeable


In [37]:
import findspark
findspark.init('/home/ubuntu/spark-3.3.1-bin-hadoop3')
findspark.find()

'/home/ubuntu/spark-3.3.1-bin-hadoop3'

In [38]:
from pyspark.sql import SparkSession

# The entry point into all functionality in Spark is the SparkSession class.
spark = (SparkSession
	.builder
	.appName("DS5110/CS5501: my awesome Spark program")
	.master("spark://172.31.24.10:7077")
	.config("spark.executor.memory", "1024M")
	.getOrCreate())

# Part 2

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import asc

In [4]:
# You can read the data from a file into DataFrames
df = spark.read.csv("hdfs://172.31.24.10:9000/export.csv")

In [5]:
sc = spark.sparkContext

In [6]:
df.head()

Row(_c0='battery_level', _c1='c02_level', _c2='cca2', _c3='cca3', _c4='cn', _c5='device_id', _c6='device_name', _c7='humidity', _c8='ip', _c9='latitude', _c10='lcd', _c11='longitude', _c12='scale', _c13='temp', _c14='timestamp')

In [7]:
df

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string, _c4: string, _c5: string, _c6: string, _c7: string, _c8: string, _c9: string, _c10: string, _c11: string, _c12: string, _c13: string, _c14: string]

In [8]:
df = (spark.read
          .format("csv")
          .option("inferSchema", True)
          .option("header", True)
          .load("hdfs://172.31.24.10:9000/export.csv"))

df

DataFrame[battery_level: int, c02_level: int, cca2: string, cca3: string, cn: string, device_id: int, device_name: string, humidity: int, ip: string, latitude: double, lcd: string, longitude: double, scale: string, temp: int, timestamp: bigint]

In [9]:
df.head()

Row(battery_level=8, c02_level=868, cca2='US', cca3='USA', cn='United States', device_id=1, device_name='meter-gauge-1xbYRYcj', humidity=51, ip='68.161.225.1', latitude=38.0, lcd='green', longitude=-97.0, scale='Celsius', temp=34, timestamp=1458444054093)

In [10]:
df.show(10)

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

In [11]:
df.printSchema()

root
 |-- battery_level: integer (nullable = true)
 |-- c02_level: integer (nullable = true)
 |-- cca2: string (nullable = true)
 |-- cca3: string (nullable = true)
 |-- cn: string (nullable = true)
 |-- device_id: integer (nullable = true)
 |-- device_name: string (nullable = true)
 |-- humidity: integer (nullable = true)
 |-- ip: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- lcd: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- scale: string (nullable = true)
 |-- temp: integer (nullable = true)
 |-- timestamp: long (nullable = true)



In [12]:
""" Here is the main part of the part 2 """
# sort the DataFrame by country code and then timestamp
sorted_df = df.orderBy(asc("cca2"), asc("timestamp"))

# show the sorted DataFrame
sorted_df.show()

+-------------+---------+----+----+--------------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|                  cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+--------------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            5|     1217|  AE| ARE|United Arab Emirates|      501|device-mac-501e4O...|      48|  213.42.16.154|    24.0|yellow|     54.0|Celsius|  16|1458444054343|
|            0|      915|  AR| ARG|           Argentina|      227|meter-gauge-2273p...|      34|  200.71.230.81|   -34.6| green|   -58.38|Celsius|  15|1458444054251|
|            1|     1189|  AR| ARG|           Argentina|      319|meter-gauge-319Y3...|      54| 200.71.236.145|   -34.6|yellow|   -58.38|Celsius|  25|1458444054287|
|   

In [13]:
# write the sorted DataFrame back to HDFS
sorted_df.write.format("csv").mode("overwrite").save("hdfs://172.31.24.10:9000/part2_output.csv")

# Part 3

## Part3 - Task 1

In [39]:
"""
This is a skeleton of the PageRank algorithm.
Feel free to use any piece of code in this provided skeleton source file.
To use it, you will need to copy it into your Notebook. 
Feel free to make modifications to template code as you see fit.
However, you are encouraged to implement the algorithm completely on
your own. :-)
"""
import re
import sys
from operator import add
from typing import Iterable, Tuple

from pyspark.resultiterable import ResultIterable
from pyspark.sql import SparkSession

In [40]:
"""Helper function to calculates URL contributions to the rank of other URLs"""
def calculateRankContrib(urls: ResultIterable[str], rank: float) -> Iterable[Tuple[str, float]]:
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)


"""Helper function to parses a urls string into urls pair"""
def parseNeighborURLs(urls: str) -> Tuple[str, str]:
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

In [41]:
# Initialize the Spark context
# TODO: You should define a new name for your PySpark PageRank program
spark = (SparkSession.builder.appName("A2:PageRank-Task1")
    .master("spark://172.31.24.10:7077")
    .config("spark.executor.memory", "2048M")
    .getOrCreate())

24/02/28 03:40:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [42]:
"""Hint: You want to use hash partitioning to optimize the performance of join"""
""" For task 1, as required, we do not use Partitioning """

# Loads in input file
#     URL         neighbor URL
#     URL         neighbor URL
#     URL         neighbor URL
#     ...
linesRDD_no_filter = spark.sparkContext.textFile("hdfs://172.31.24.10:9000/web-BerkStan.txt")
# exclude the the empty rows starts with #
linesRDD = linesRDD_no_filter.filter(lambda line : not line.startswith('#'))

# Perform a transformation to define a links RDD by using parseNeighborURLs helper function
linksRDD = linesRDD.map(lambda urls: parseNeighborURLs(urls)).distinct().groupByKey()

# Initialize a ranks RDD
# as required, we add persist() to persist the ranks RDD in memory or disk
ranksRDD = linksRDD.map(lambda url_neighbors: (url_neighbors[0], 1.0)).persist()

### ----------------------------------------------------------------------------------------------------------------------------------------------- Following section is for testing and getting familiar with the data and spark, if you are grading my assignment, please just ignore this section

In [33]:
linksRDD

PythonRDD[92] at RDD at PythonRDD.scala:53

In [34]:
ranksRDD

PythonRDD[93] at RDD at PythonRDD.scala:53

In [39]:
table_join = linksRDD.join(ranksRDD)

In [42]:
sample_data = table_join.take(10)

In [43]:
for data in sample_data:
    print(data)

('722', (<pyspark.resultiterable.ResultIterable object at 0x7fa1c7a08520>, 1.0))
('77', (<pyspark.resultiterable.ResultIterable object at 0x7fa1c7a083a0>, 1.0))
('143', (<pyspark.resultiterable.ResultIterable object at 0x7fa1c7a08700>, 1.0))
('256', (<pyspark.resultiterable.ResultIterable object at 0x7fa1c7a080d0>, 1.0))
('330', (<pyspark.resultiterable.ResultIterable object at 0x7fa1c7a08250>, 1.0))
('483', (<pyspark.resultiterable.ResultIterable object at 0x7fa1c7a08100>, 1.0))
('61273', (<pyspark.resultiterable.ResultIterable object at 0x7fa1c7a082b0>, 1.0))
('801', (<pyspark.resultiterable.ResultIterable object at 0x7fa1c7a08220>, 1.0))
('158745', (<pyspark.resultiterable.ResultIterable object at 0x7fa1c7a081f0>, 1.0))
('172984', (<pyspark.resultiterable.ResultIterable object at 0x7fa1c7a08df0>, 1.0))


In [18]:
linesRDD.getNumPartitions()

2

In [19]:
linksRDD.getNumPartitions()

2

In [20]:
ranksRDD.getNumPartitions()

2

### ----------------------------------------------------------------------------------------------------------------------------------------------- my testing ends here, following are my assignment code for grading

In [43]:
# Calculates and updates URL ranks continuously using PageRank algorithm.
# Replace N with a number ranging from 3 to 10
# For A2 you are required to complete 10 iterations
N = 10
for iteration in range(N):
    # TODO: Implement the PageRank algorithm here
    # TODO: Implement the PageRank algorithm here
    # TODO: Implement the PageRank algorithm here
    
    # calculate URL contributions to the rank of other URLs
    contributionsRDD = linksRDD.join(ranksRDD).flatMap(lambda x: calculateRankContrib(x[1][0], x[1][1]))

    # Update URL ranks using the PageRank algorithm
    # as required, we add persist() to persist the ranks RDD in memory or disk
    ranksRDD = contributionsRDD.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank).persist()


In [44]:
# Show the first 50 rows with the highest ranks
top_50_ranks = ranksRDD.takeOrdered(50, key=lambda x: -x[1])
print("The first 50 rows with the highest ranks:")
for url, rank in top_50_ranks:
    print("{}: {}".format(url, rank))

The first 50 rows with the highest ranks:
272919: 6531.324623752456
438238: 4335.323158564437
571448: 2383.89760741189
601656: 2195.3940755967287
316792: 1855.6908757901515
319209: 1632.8193684975693
184094: 1532.284237448339
571447: 1492.930163093879
401873: 1436.1600933469288
66244: 1261.578395867334
68949: 1260.7919421349131
284306: 1257.2475650644853
68948: 1251.1723536459224
96070: 1235.2985405976256
95552: 1235.2985405976253
86238: 1235.2985405976253
77284: 1235.2985405976253
68946: 1235.2985405976253
86239: 1235.2985405976253
66909: 1235.2985405976253
86237: 1235.2985405976253
95551: 1235.2985405976253
68947: 1235.2985405976253
768: 1225.5975665113076
927: 1117.8383051141836
210376: 920.6701252803681
95527: 919.6797146521095
100130: 916.0190658202691
101163: 912.5380530105953
95018: 911.1831080077993
100646: 909.7095673033018
96045: 904.3981315809754
66879: 895.7909746044769
210305: 893.0386730972408
319412: 887.9352083382672
571451: 875.7852546255615
570985: 871.5825582573231
5

In [23]:
# Save output to HDFS
ranksDf = ranksRDD.toDF()
# Note: You should not overwrite the input file. Otherwise you need to reload it
ranksDf.write.format("csv").mode("overwrite").save("hdfs://172.31.24.10:9000/part3_task1_output.csv")

In [36]:
# (Optionally) stop the Spark session when it's done
# spark.stop()

## Part 3 - Task 2

### task2-1 Try to persist every iteration

In [54]:
# Initialize the Spark context
# TODO: You should define a new name for your PySpark PageRank program
spark = (SparkSession.builder.appName("A2:PageRank-Task2")
    .master("spark://172.31.24.10:7077")
    .config("spark.executor.memory", "2048M")
    .getOrCreate())

24/02/28 04:25:56 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [55]:
"""Hint: You want to use hash partitioning to optimize the performance of join"""
""" For task 2, as required, we will use Partitioning, by using partitionBy() method """

# Loads in input file
#     URL         neighbor URL
#     URL         neighbor URL
#     URL         neighbor URL
#     ...
linesRDD_no_filter = spark.sparkContext.textFile("hdfs://172.31.24.10:9000/web-BerkStan.txt")
linesRDD = linesRDD_no_filter.filter(lambda line : not line.startswith('#'))

# the number of partitions, task 1 is 2, so in task 2, we will try 4
partition_N = 4

# Perform a transformation to define a links RDD by using parseNeighborURLs helper function
# we partition the links RDD into N parts
linksRDD = linesRDD.map(lambda urls: parseNeighborURLs(urls)).distinct().groupByKey().partitionBy(partition_N)

# Initialize a ranks RDD
# we partition the ranks RDD into N parts
# and as required, we add persist() to persist the ranks RDD in memory or disk
ranksRDD = linksRDD.map(lambda url_neighbors: (url_neighbors[0], 1.0)).partitionBy(partition_N).persist()


In [56]:
# Calculates and updates URL ranks continuously using PageRank algorithm.
# Replace N with a number ranging from 3 to 10
# For A2 you are required to complete 10 iterations
N = 10
for iteration in range(N):
    # TODO: Implement the PageRank algorithm here
    # TODO: Implement the PageRank algorithm here
    # TODO: Implement the PageRank algorithm here
    
    # calculate URL contributions to the rank of other URLs
    contributionsRDD = linksRDD.join(ranksRDD).flatMap(lambda x: calculateRankContrib(x[1][0], x[1][1]))

    # Update URL ranks using the PageRank algorithm
    # as required, we add persist() to persist the ranks RDD in memory or disk
    ranksRDD = contributionsRDD.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank).persist()


In [48]:
# Show the first 50 rows with the highest ranks
top_50_ranks = ranksRDD.takeOrdered(50, key=lambda x: -x[1])
print("The first 50 rows with the highest ranks:")
for url, rank in top_50_ranks:
    print("{}: {}".format(url, rank))

The first 50 rows with the highest ranks:
272919: 6531.324623752462
438238: 4335.323158564417
571448: 2383.897607411887
601656: 2195.3940755967305
316792: 1855.6908757901547
319209: 1632.8193684975668
184094: 1532.2842374483457
571447: 1492.9301630938799
401873: 1436.1600933469324
66244: 1261.57839586733
68949: 1260.791942134909
284306: 1257.247565064486
68948: 1251.1723536459187
86237: 1235.2985405976215
77284: 1235.2985405976215
95551: 1235.2985405976215
66909: 1235.2985405976212
86238: 1235.2985405976212
96070: 1235.2985405976212
86239: 1235.298540597621
68946: 1235.298540597621
68947: 1235.2985405976206
95552: 1235.2985405976206
768: 1225.5975665113092
927: 1117.838305114185
210376: 920.670125280368
95527: 919.6797146521053
100130: 916.019065820265
101163: 912.538053010591
95018: 911.1831080077952
100646: 909.709567303298
96045: 904.3981315809713
66879: 895.7909746044725
210305: 893.0386730972408
319412: 887.935208338267
571451: 875.7852546255614
570985: 871.582558257324
544858: 86

### task2-2 Try to persist every 2 iterations

In [50]:
# Initialize the Spark context
# TODO: You should define a new name for your PySpark PageRank program
spark = (SparkSession.builder.appName("A2:PageRank-Task2-2")
    .master("spark://172.31.24.10:7077")
    .config("spark.executor.memory", "2048M")
    .getOrCreate())

24/02/28 04:00:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [51]:
"""Hint: You want to use hash partitioning to optimize the performance of join"""
""" For task 2, as required, we will use Partitioning, by using partitionBy() method """
""" This time, we try to not call persist() every iteration, rather every 2 iterations """

# Loads in input file
#     URL         neighbor URL
#     URL         neighbor URL
#     URL         neighbor URL
#     ...
linesRDD_no_filter = spark.sparkContext.textFile("hdfs://172.31.24.10:9000/web-BerkStan.txt")
linesRDD = linesRDD_no_filter.filter(lambda line : not line.startswith('#'))

# the number of partitions, task 1 is 2, so in task 2, we will try 4
partition_N = 4

# Perform a transformation to define a links RDD by using parseNeighborURLs helper function
# we partition the links RDD into N parts
linksRDD = linesRDD.map(lambda urls: parseNeighborURLs(urls)).distinct().groupByKey().partitionBy(partition_N)

# Initialize a ranks RDD
# we partition the ranks RDD into N parts
# and as required, we add persist() to persist the ranks RDD in memory or disk
ranksRDD = linksRDD.map(lambda url_neighbors: (url_neighbors[0], 1.0)).partitionBy(partition_N).persist()


In [52]:
# Calculates and updates URL ranks continuously using PageRank algorithm.
# Replace N with a number ranging from 3 to 10
# For A2 you are required to complete 10 iterations
N = 10

for iteration in range(N):
    # TODO: Implement the PageRank algorithm here
    # TODO: Implement the PageRank algorithm here
    # TODO: Implement the PageRank algorithm here
    
    # calculate URL contributions to the rank of other URLs
    contributionsRDD = linksRDD.join(ranksRDD).flatMap(lambda x: calculateRankContrib(x[1][0], x[1][1]))

    # Update URL ranks using the PageRank algorithm
    # as required, we add persist() to persist the ranks RDD in memory or disk
    """ persist every 2 iterations """
    if not iteration % 2:
        ranksRDD = contributionsRDD.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank).persist()
    else:
        ranksRDD = contributionsRDD.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank)


In [53]:
# Show the first 50 rows with the highest ranks
top_50_ranks = ranksRDD.takeOrdered(50, key=lambda x: -x[1])
print("The first 50 rows with the highest ranks:")
for url, rank in top_50_ranks:
    print("{}: {}".format(url, rank))

The first 50 rows with the highest ranks:
272919: 6531.324623752462
438238: 4335.323158564447
571448: 2383.897607411887
601656: 2195.3940755967305
316792: 1855.6908757901526
319209: 1632.8193684975718
184094: 1532.2842374483391
571447: 1492.9301630938783
401873: 1436.1600933469294
66244: 1261.5783958673344
68949: 1260.7919421349134
284306: 1257.2475650644838
68948: 1251.1723536459226
66909: 1235.2985405976262
95551: 1235.298540597626
96070: 1235.2985405976258
68947: 1235.2985405976249
95552: 1235.2985405976249
86237: 1235.2985405976249
86239: 1235.2985405976249
68946: 1235.2985405976249
86238: 1235.2985405976249
77284: 1235.2985405976249
768: 1225.5975665113071
927: 1117.8383051141816
210376: 920.6701252803678
95527: 919.6797146521111
100130: 916.0190658202707
101163: 912.538053010597
95018: 911.1831080078009
100646: 909.7095673033035
96045: 904.3981315809768
66879: 895.7909746044784
210305: 893.038673097241
319412: 887.935208338267
571451: 875.7852546255582
570985: 871.5825582573211
5

## Part 3 - Task 3

### kill at arounf 65% of the progress

In [57]:
# Initialize the Spark context
# TODO: You should define a new name for your PySpark PageRank program
spark = (SparkSession.builder.appName("A2:PageRank-Task3")
    .master("spark://172.31.24.10:7077")
    .config("spark.executor.memory", "2048M")
    .getOrCreate())

24/02/28 04:26:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [58]:
"""Hint: You want to use hash partitioning to optimize the performance of join"""
""" For task 3, we will use the scenario used in task2 with persist for every iteration """

# Loads in input file
#     URL         neighbor URL
#     URL         neighbor URL
#     URL         neighbor URL
#     ...
linesRDD_no_filter = spark.sparkContext.textFile("hdfs://172.31.24.10:9000/web-BerkStan.txt")
linesRDD = linesRDD_no_filter.filter(lambda line : not line.startswith('#'))

# the number of partitions, task 1 is 2, so in task 2, we will try 4
partition_N = 4

# Perform a transformation to define a links RDD by using parseNeighborURLs helper function
# we partition the links RDD into N parts
linksRDD = linesRDD.map(lambda urls: parseNeighborURLs(urls)).distinct().groupByKey().partitionBy(partition_N)

# Initialize a ranks RDD
# we partition the ranks RDD into N parts
# and as required, we add persist() to persist the ranks RDD in memory or disk
ranksRDD = linksRDD.map(lambda url_neighbors: (url_neighbors[0], 1.0)).partitionBy(partition_N).persist()


In [59]:
# Calculates and updates URL ranks continuously using PageRank algorithm.
# Replace N with a number ranging from 3 to 10
# For A2 you are required to complete 10 iterations
N = 10
for iteration in range(N):
    # TODO: Implement the PageRank algorithm here
    # TODO: Implement the PageRank algorithm here
    # TODO: Implement the PageRank algorithm here
    
    # calculate URL contributions to the rank of other URLs
    contributionsRDD = linksRDD.join(ranksRDD).flatMap(lambda x: calculateRankContrib(x[1][0], x[1][1]))

    # Update URL ranks using the PageRank algorithm
    # as required, we add persist() to persist the ranks RDD in memory or disk
    ranksRDD = contributionsRDD.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: 0.15 + 0.85 * rank).persist()


In [60]:
# Show the first 50 rows with the highest ranks
top_50_ranks = ranksRDD.takeOrdered(50, key=lambda x: -x[1])
print("The first 50 rows with the highest ranks:")
for url, rank in top_50_ranks:
    print("{}: {}".format(url, rank))

24/02/28 04:31:11 ERROR TaskSchedulerImpl: Lost executor 1 on 172.31.27.103: worker lost
24/02/28 04:31:11 WARN TaskSetManager: Lost task 2.0 in stage 66.0 (TID 442) (172.31.27.103 executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: worker lost
24/02/28 04:31:11 WARN TaskSetManager: Lost task 0.0 in stage 66.0 (TID 440) (172.31.27.103 executor 1): ExecutorLostFailure (executor 1 exited caused by one of the running tasks) Reason: worker lost
24/02/28 04:31:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_584_2 !
24/02/28 04:31:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_381_2 !
24/02/28 04:31:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_336_2 !
24/02/28 04:31:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_381_0 !
24/02/28 04:31:11 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_197_2 !
24/02/28 04:31:11 WARN BlockManagerMaster

24/02/28 04:31:15 WARN TaskSetManager: Lost task 0.1 in stage 66.0 (TID 444) (172.31.24.10 executor 0): FetchFailed(BlockManagerId(1, 172.31.27.103, 34495, None), shuffleId=61, mapIndex=0, mapId=398, reduceId=0, message=
org.apache.spark.shuffle.FetchFailedException
	at org.apache.spark.errors.SparkCoreErrors$.fetchFailedError(SparkCoreErrors.scala:312)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.scala:1166)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:904)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.next(ShuffleBlockFetcherIterator.scala:85)
	at org.apache.spark.util.CompletionIterator.next(CompletionIterator.scala:29)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.util.CompletionIter

The first 50 rows with the highest ranks:
272919: 6531.324623752464
438238: 4335.323158564445
571448: 2383.8976074118905
601656: 2195.394075596731
316792: 1855.6908757901533
319209: 1632.8193684975713
184094: 1532.2842374483419
571447: 1492.9301630938785
401873: 1436.1600933469288
66244: 1261.5783958673337
68949: 1260.7919421349134
284306: 1257.2475650644833
68948: 1251.172353645923
96070: 1235.298540597627
66909: 1235.2985405976258
95552: 1235.2985405976253
86237: 1235.2985405976253
86239: 1235.2985405976253
68946: 1235.2985405976253
86238: 1235.2985405976253
68947: 1235.298540597625
95551: 1235.2985405976237
77284: 1235.2985405976235
768: 1225.5975665113065
927: 1117.8383051141816
210376: 920.6701252803681
95527: 919.6797146521109
100130: 916.0190658202705
101163: 912.5380530105969
95018: 911.1831080078007
100646: 909.7095673033034
96045: 904.3981315809767
66879: 895.7909746044783
210305: 893.0386730972414
319412: 887.9352083382672
571451: 875.7852546255579
570985: 871.582558257325
5